## Bagging (bootstrapping)

In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
# 데이터 불러오기
data = pd.read_csv("./data/kc_house_data.csv") 
data.head()

,id,date,price,bedrooms,bathrooms,floors,waterfront,condition,grade,yr_built,yr_renovated,zipcode,lat,long
0,7129300520,20141013T000000,221900.0,3,1.00,1.0,0,3,7,1955,0,98178,47.5112,-122.257
1,6414100192,20141209T000000,538000.0,3,2.25,2.0,0,3,7,1951,1991,98125,47.7210,-122.319
2,5631500400,20150225T000000,180000.0,2,1.00,1.0,0,3,6,1933,0,98028,47.7379,-122.233
3,2487200875,20141209T000000,604000.0,4,3.00,1.0,0,5,7,1965,0,98136,47.5208,-122.393
4,1954400510,20150218T000000,510000.0,3,2.00,1.0,0,3,8,1987,0,98074,47.6168,-122.045


In [3]:
'''
id: 집 고유아이디
date: 집이 팔린 날짜 
price: 집 가격 (타겟변수)
bedrooms: 주택 당 침실 개수
bathrooms: 주택 당 화장실 개수
floors: 전체 층 개수
waterfront: 해변이 보이는지 (0, 1)
condition: 집 청소상태 (1~5)
grade: King County grading system 으로 인한 평점 (1~13)
yr_built: 집이 지어진 년도
yr_renovated: 집이 리모델링 된 년도
zipcode: 우편번호
lat: 위도
long: 경도
'''

'\nid: 집 고유아이디\ndate: 집이 팔린 날짜 \nprice: 집 가격 (타겟변수)\nbedrooms: 주택 당 침실 개수\nbathrooms: 주택 당 화장실 개수\nfloors: 전체 층 개수\nwaterfront: 해변이 보이는지 (0, 1)\ncondition: 집 청소상태 (1~5)\ngrade: King County grading system 으로 인한 평점 (1~13)\nyr_built: 집이 지어진 년도\nyr_renovated: 집이 리모델링 된 년도\nzipcode: 우편번호\nlat: 위도\nlong: 경도\n'

In [4]:
nCar = data.shape[0]
nVar = data.shape[1]
print('nCar: %d' % nCar, 'nVar: %d' % nVar )

nCar: 21613 nVar: 14


## 의미가 없다고 판단되는 변수 제거

In [5]:
data = data.drop(['id', 'date', 'zipcode', 'lat', 'long'], axis = 1)

## 범주형 변수를 이진형 변수로 변환
- 범주형 변수는 waterfront 컬럼 뿐이며, 이진 분류이기 때문에 0, 1로 표현한다.
- 데이터에서 0, 1로 표현되어 있으므로 과정 생략

## 설명변수와 타겟변수를 분리, 학습데이터와 평가데이터 분리

In [6]:
feature_columns = list(data.columns.difference(['price'])) # Price를 제외한 모든 행
X = data[feature_columns]
y = data['price']
train_x, test_x, train_y, test_y = train_test_split(X, y, test_size = 0.3, random_state = 42) # 학습데이터와 평가데이터의 비율을 7:3
print(train_x.shape, test_x.shape, train_y.shape, test_y.shape) # 데이터 개수 확인

(15129, 8) (6484, 8) (15129,) (6484,)


## 학습 데이터를 선형 회귀 모형에 적합 후 평가 데이터로 검증 (Stats_Models)

In [7]:
import statsmodels.api as sm
from sklearn.metrics import mean_squared_error, r2_score
from math import sqrt

sm_train_x = sm.add_constant(train_x, has_constant = 'add') # Bias 추가
sm_model = sm.OLS(train_y, sm_train_x) # 모델 구축
fitted_sm_model = sm_model.fit() # 학습 진행
fitted_sm_model.summary() # 학습 모델 구조 확인

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.595
Model:                            OLS   Adj. R-squared:                  0.595
Method:                 Least Squares   F-statistic:                     2776.
Date:                Thu, 30 Jul 2020   Prob (F-statistic):               0.00
Time:                        17:36:25   Log-Likelihood:            -2.0826e+05
No. Observations:               15129   AIC:                         4.165e+05
Df Residuals:                   15120   BIC:                         4.166e+05
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const         7.186e+06   1.73e+05     41.548      0.000    6.85e+06    7.52e+06
bathrooms     1.303e+05   3960.833     32.889      0.000    1.23e+05    1.38e+05
bedrooms     -2224.7910   2382.356     -0.934      0.350   -6894.497    2444.915
condition     1.641e+04   3169.013      5.178      0.000    1.02e+04    2.26e+04
floors        1946.3052   4336.838      0.449      0.654   -6554.422    1.04e+04
grade         1.956e+05   2199.540     88.924      0.000    1.91e+05       2e+05
waterfront    7.555e+05   2.26e+04     33.479      0.000    7.11e+05       8e+05
yr_built     -4300.7865     88.073    -48.832      0.000   -4473.420   -4128.153
yr_renovated    12.7325      5.043      2.525      0.012       2.847      22.618
==============================================================================
Omnibus:                    13447.374   Durbin-Watson:                   1.994
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          1684794.827
Skew:                           3.763   Prob(JB):                         0.00
Kurtosis:                      54.147   Cond. No.                     1.82e+05
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.82e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [8]:
# 결과 확인
sm_test_x = sm.add_constant(test_x, has_constant = 'add') # 테스트 데이터에 Bias 추가
sm_model_predict = fitted_sm_model.predict(sm_test_x) # 테스트 데이터 예측
print("RMSE: {}\n".format(sqrt(mean_squared_error(sm_model_predict, test_y)))) # RMSE
print(fitted_sm_model.params) # 회귀계수

RMSE: 239804.2967085816

const           7.185671e+06
bathrooms       1.302689e+05
bedrooms       -2.224791e+03
condition       1.641020e+04
floors          1.946305e+03
grade           1.955909e+05
waterfront      7.555423e+05
yr_built       -4.300787e+03
yr_renovated    1.273246e+01
dtype: float64


## Bagging 한 결과가 일반적인 결과보다 좋은지 확인

In [9]:
import random

bagging_predict_result = [] # 빈 리스트 생성

for _ in range(10):
    data_index = [data_index for data_index in range(train_x.shape[0])] # 학습 데이터의 인덱스를 리스트로 변환
    random_data_index = np.random.choice(data_index, train_x.shape[0]) # 데이터의 1/10 크기만큼 랜덤 샘플링, // 는 소수점을 무시하기 위함
    print(len(set(random_data_index)))
    sm_train_x = train_x.iloc[random_data_index, ] # 랜덤 인덱스에 해당되는 학습 데이터 중 설명 변수
    sm_train_y = train_y.iloc[random_data_index, ] # 랜덤 인덱스에 해당되는 학습 데이터 중 종속 변수
    sm_train_x = sm.add_constant(sm_train_x, has_constant = 'add') # Bias 추가
    sm_model = sm.OLS(sm_train_y, sm_train_x) # 모델 구축
    fitted_sm_model = sm_model.fit() # 학습 진행
    
    sm_test_x = sm.add_constant(test_x, has_constant = 'add') # 테스트 데이터에 Bias 추가
    sm_model_predict = fitted_sm_model.predict(sm_test_x) # 테스트 데이터 예측
    bagging_predict_result.append(sm_model_predict) # 반복문이 실행되기 전 빈 리스트에 결과 값 저장

9587
9576
9593
9571
9540
9575
9635
9571
9577
9592


In [10]:
bagging_predict_result[0] # 0 ~ 9, 10번의 예측을 하였기 때문에 10개의 결과가 생성

735      5.640296e+05
2830     7.184781e+05
4106     1.121120e+06
16218    1.487513e+06
19964    7.047517e+05
             ...     
12606    6.064193e+05
14393    6.801028e+05
6899     3.218954e+05
85       9.039520e+05
21363    4.374061e+05
Length: 6484, dtype: float64

In [11]:
# Bagging을 바탕으로 예측한 결과값에 대한 평균을 계산
bagging_predict = [] # 빈 리스트 생성
for lst2_index in range(test_x.shape[0]): # 테스트 데이터 개수만큼의 반복
    temp_predict = [] # 임시 빈 리스트 생성 (반복문 내 결과값 저장)
    for lst_index in range(len(bagging_predict_result)): # Bagging 결과 리스트 반복
        temp_predict.append(bagging_predict_result[lst_index].values[lst2_index]) # 각 Bagging 결과 예측한 값 중 같은 인덱스를 리스트에 저장
    bagging_predict.append(np.mean(temp_predict)) # 해당 인덱스의 30개의 결과값에 대한 평균을 최종 리스트에 추가

In [12]:
# 예측한 결과값들의 평균을 계산하여 실제 테스트 데이트의 타겟변수와 비교하여 성능 평가
print("RMSE: {}".format(sqrt(mean_squared_error(bagging_predict, test_y)))) # RMSE

RMSE: 239771.0397124748


## 학습 데이터를 선형 회귀 모형에 적합 후 평가 데이터로 검증 (Scikit-Learn)

In [13]:
from sklearn.linear_model import LinearRegression
regression_model = LinearRegression() # 선형 회귀 모형
linear_model1 = regression_model.fit(train_x, train_y) # 학습 데이터를 선형 회귀 모형에 적합
predict1 = linear_model1.predict(test_x) # 학습된 선형 회귀 모형으로 평가 데이터 예측
print("RMSE: {}".format(sqrt(mean_squared_error(predict1, test_y)))) # RMSE 결과

RMSE: 239804.29670858145


## Bagging 을 이용하여 선형 회귀 모형에 적합 후 평가 (Sampling 10번)

In [14]:
from sklearn.ensemble import BaggingRegressor
bagging_model = BaggingRegressor(base_estimator = regression_model, # 선형회귀모형
                                 n_estimators = 5, # 5번 샘플링
                                 verbose = 1) # 학습 과정 표시
linear_model2 = bagging_model.fit(train_x, train_y) # 학습 진행
predict2 = linear_model2.predict(test_x) # 학습된 Bagging 선형 회귀 모형으로 평가 데이터 예측
print("RMSE: {}".format(sqrt(mean_squared_error(predict2, test_y)))) # RMSE 결과

RMSE: 239936.4471006533


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished


## 그렇다면 Sampling을 많이 해보자!

In [15]:
bagging_model2 = BaggingRegressor(base_estimator = regression_model, # 선형 회귀모형
                                  n_estimators = 30, # 30번 샘플링
                                  verbose = 1) # 학습 과정 표시
linear_model3 = bagging_model2.fit(train_x, train_y) # 학습 진행
predict3 = linear_model3.predict(test_x) # 학습된 Bagging 선형 회귀 모형으로 평가 데이터 예측
print("RMSE: {}".format(sqrt(mean_squared_error(predict3, test_y)))) # RMSE 결과

RMSE: 239838.5686255727


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished


## 학습 데이터를 의사결정나무모형에 적합 후 평가 데이터로 검증

In [16]:
from sklearn.tree import DecisionTreeRegressor
decision_tree_model = DecisionTreeRegressor() # 의사결정나무 모형
tree_model1 = decision_tree_model.fit(train_x, train_y) # 학습 데이터를 의사결정나무 모형에 적합
predict1 = tree_model1.predict(test_x) # 학습된 의사결정나무 모형으로 평가 데이터 예측
print("RMSE: {}".format(sqrt(mean_squared_error(predict1, test_y)))) # RMSE 결과

RMSE: 284298.1676879424


In [17]:
import random
bagging_predict_result = [] # 빈 리스트 생성
for _ in range(30):
    data_index = [data_index for data_index in range(train_x.shape[0])] # 학습 데이터의 인덱스를 리스트로 변환
    random_data_index = np.random.choice(data_index, train_x.shape[0]) # 데이터의 1/10 크기만큼 랜덤 샘플링, // 는 소수점을 무시하기 위함
    print(len(set(random_data_index)))
    sm_train_x = train_x.iloc[random_data_index, ] # 랜덤 인덱스에 해당되는 학습 데이터 중 설명 변수
    sm_train_y = train_y.iloc[random_data_index, ] # 랜덤 인덱스에 해당되는 학습 데이터 중 종속 변수
    decision_tree_model = DecisionTreeRegressor() # 의사결정나무 모형
    tree_model1 = decision_tree_model.fit(sm_train_x, sm_train_y) # 학습 데이터를 의사결정나무 모형에 적합
 
    predict1 = tree_model1.predict(test_x) # 테스트 데이터 예측
    bagging_predict_result.append(predict1) # 반복문이 실행되기 전 빈 리스트에 결과 값 저장

9582
9551
9579
9521
9643
9552
9550
9551
9562
9483
9567
9552
9576
9559
9571
9520
9671
9516
9578
9579
9531
9494
9548
9545
9550
9576
9580
9583
9533
9633


In [18]:
# Bagging을 바탕으로 예측한 결과값에 대한 평균을 계산
bagging_predict = [] # 빈 리스트 생성
for lst2_index in range(test_x.shape[0]): # 테스트 데이터 개수만큼의 반복
    temp_predict = [] # 임시 빈 리스트 생성 (반복문 내 결과값 저장)
    for lst_index in range(len(bagging_predict_result)): # Bagging 결과 리스트 반복
        temp_predict.append(bagging_predict_result[lst_index][lst2_index]) # 각 Bagging 결과 예측한 값 중 같은 인덱스를 리스트에 저장
    bagging_predict.append(np.mean(temp_predict)) # 해당 인덱스의 30개의 결과값에 대한 평균을 최종 리스트에 추가

In [19]:
# 예측한 결과값들의 평균을 계산하여 실제 테스트 데이트의 타겟변수와 비교하여 성능 평가

print("RMSE: {}".format(sqrt(mean_squared_error(bagging_predict, test_y)))) # RMSE

RMSE: 237399.69910405998


In [20]:
bagging_predict_result[29]

array([322500.        , 720000.        , 925666.66666667, ...,
       266196.08695652, 605004.        , 305000.        ])

## Bagging 을 이용하여 의사결정나무모형에 적합 후 평가 (Sampling 10번)

In [21]:
bagging_decision_tree_model1 = BaggingRegressor(base_estimator = decision_tree_model, # 의사결정나무 모형
                                                n_estimators = 5, # 5번 샘플링
                                                verbose = 1) # 학습 과정 표시
tree_model2 = bagging_decision_tree_model1.fit(train_x, train_y) # 학습 진행
predict2 = tree_model2.predict(test_x) # 학습된 Bagging 의사결정나무 모형으로 평가 데이터 예측
print("RMSE: {}".format(sqrt(mean_squared_error(predict2, test_y)))) # RMSE 결과

RMSE: 243523.94778527206


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished


In [22]:
bagging_decision_tree_model2 = BaggingRegressor(base_estimator = decision_tree_model, # 의사결정나무 모형
                                                n_estimators = 30, # 30번 샘플링
                                                verbose = 1) # 학습 과정 표시
tree_model3 = bagging_decision_tree_model2.fit(train_x, train_y) # 학습 진행
predict3 = tree_model3.predict(test_x) # 학습된 Bagging 의사결정나무 모형으로 평가 데이터 예측
print("RMSE: {}".format(sqrt(mean_squared_error(predict3, test_y)))) # RMSE 결과

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


RMSE: 233091.59790161543


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.5s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished
